In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Forces synchronous CUDA errors
os.environ["TORCH_USE_CUDA_DSA"] = "1"    # Enables device-side assert tracking

In [3]:
!git clone https://github.com/microsoft/LLaVA-Med.git


Cloning into 'LLaVA-Med'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 451 (delta 53), reused 51 (delta 49), pack-reused 345 (from 2)
Receiving objects: 100% (451/451), 77.08 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (135/135), done.
Filtering content: 100% (10/10), 813.63 MiB | 28.55 MiB/s, done.


In [4]:
%cd LLaVA-Med

/content/LLaVA-Med


In [1]:
!pip install --upgrade pip setuptools wheel
!pip install -e .
!pip install numpy==1.26.4
!pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.36.2 peft==0.4.0 accelerate==0.21.0


Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 192.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 142.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 159.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 

In [1]:
%cd LLaVA-Med
!mkdir -p checkpoints
!huggingface-cli download microsoft/llava-med-v1.5-mistral-7b --local-dir checkpoints/llava-med-v1.5-mistral-7b --local-dir-use-symlinks False


/content/LLaVA-Med
/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]Downloading 'model-00001-of-00004.safetensors' to 'checkpoints/llava-med-v1.5-mistral-7b/.cache/huggingface/download/IO4xwqmZYzFmxznkwkiNSBwO1H0=.ef2190dc6c2a940e60f03f5fdb4dddb2320eb87801aeca5c40b0a28ce8aa420e.incomplete'

model-00001-of-00004.safetensors:   0% 0.00/4.94G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0% 0.00/262M [00:00<?, ?B/s]


model-00002-of-00004.safetensors:   0% 0.00/5.00G [00:00<?, ?B/s]Downloading 'generation_config.json' to 'checkpoints/llava-med-v1.5-mistral-7b/.cache/huggingface/download/3EVKVggOldJcKSsGjSdoUCN1AyQ=.c533f934c6359393a56a3ea067a0df118c14797e.incomplete'

model-00001-of-00004.safetensors:   0% 10.5M/4.94G [00:00<00:49, 98.8MB/s]



model-00003-of-00004.safetensors

# Load Base Model

In [1]:
%cd LLaVA-Med

/content/LLaVA-Med


In [2]:
from torch.utils.data import Dataset
from PIL import Image
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from llava.model.builder import load_pretrained_model
import pandas as pd
import os
import torch

# Step 1: Load pretrained LLaVA-Med model
tokenizer, base_model, image_processor, context_len = load_pretrained_model(
    model_path="checkpoints/llava-med-v1.5-mistral-7b",
    model_base=None,
    model_name="llava-med-v1.5-mistral-7b"
)

# Step 2: Ensure tokenizer has a valid pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.unk_token

# Step 3: Define LoRA config with BitFit (bias="all")
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="all",  # Change if BitFit or Not
    task_type="CAUSAL_LM"
)

# Step 4: Apply LoRA + BitFit to base model
model = get_peft_model(base_model, lora_config)

# Step 5: Unfreeze LayerNorm parameters (optional but recommended)
for name, param in model.named_parameters():
    if any(norm in name.lower() for norm in ["norm", "ln"]):
        param.requires_grad = True

for name, param in model.named_parameters():
    if param.requires_grad and not name.endswith(".lora_A") and not name.endswith(".lora_B"):
        param.data = param.data.to(torch.float32)

# Step 6: Report trainable parameters
model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at checkpoints/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.5.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.self_attn.out_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.10.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.20.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.7.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.post_layernorm.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.14.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.7.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.self_attn.k_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.8.self_attn.k_proj.weight', 'model.vision_

trainable params: 44,900,352 || all params: 7,610,521,600 || trainable%: 0.5899773282293818


# Load PEFT Model

In [ ]:
# @title Default title text
from torch.utils.data import Dataset
from PIL import Image
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path
from transformers import TrainingArguments, Trainer, AutoTokenizer
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel
from llava.model.builder import load_pretrained_model
import pandas as pd
import os
import torch

model_file_path = "/content/drive/MyDrive/lora_output/model (BitFit 2 Epoch)"
# Load PEFT configuration
peft_config = PeftConfig.from_pretrained(model_file_path)

# Infer model name
model_name = get_model_name_from_path(peft_config.base_model_name_or_path)

tokenizer, base_model, image_processor, context_len = load_pretrained_model(
    model_path=peft_config.base_model_name_or_path,
    model_base=None,
    model_name=model_name,
    load_8bit=False,
    load_4bit=False,
)

# Load the LoRA adapter weights
model = PeftModel.from_pretrained(base_model, model_file_path)

# Step 5: Unfreeze LayerNorm parameters (optional but recommended)
for name, param in model.named_parameters():
    if any(norm in name.lower() for norm in ["norm", "ln"]):
        param.requires_grad = True

for name, param in model.named_parameters():
    if param.requires_grad and not name.endswith(".lora_A") and not name.endswith(".lora_B"):
        param.data = param.data.to(torch.float32)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at checkpoints/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.13.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.18.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.self_attn.out_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.self_attn.k_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.16.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.mlp.fc2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.19.mlp.fc2.weight', 'model.

pytorch_model.bin:  47%|####7     | 1.54G/3.25G [00:00<?, ?B/s]

# Training

In [3]:
class MultiClassChestXrayDataset(Dataset):
    def __init__(self, image_dir, csv_path, metadata_csv, tokenizer, image_processor):
        self.data = pd.read_csv(csv_path)
        self.metadata = pd.read_csv(metadata_csv).set_index("Image Index")[["Patient Age", "Patient Sex"]].to_dict(orient="index")
        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.image_processor = image_processor

        # Label mapping
        self.label_to_condition = {
            0: "No Finding",
            1: "Pneumothorax",
            2: "Cardiomegaly",
            3: "Infiltration",
            4: "Effusion",
            5: "Emphysema",
            6: "Atelectasis"
        }

        # Filter out missing images
        self.data = self.data[self.data.iloc[:, 0].apply(lambda f: os.path.exists(os.path.join(image_dir, f)))].reset_index(drop=True)
        print(f"Dataset initialized with {len(self.data)} valid samples")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = self.data.iloc[idx, 0]
        label = int(self.data.iloc[idx, 1])

        meta = self.metadata.get(filename, {})
        age = meta.get('Patient Age', 'unknown')
        sex = meta.get('Patient Sex', 'unknown')

        # Create the conversation with multiple choice format
        question = f"You are an expert radiologist. This chest X-ray is from a {age}-year-old {sex} patient. What condition does this chest X-ray show? Choose from: 'No Finding', 'Pneumothorax', 'Cardiomegaly', 'Infiltration', 'Effusion', 'Emphysema', or 'Atelectasis'."
        answer = self.label_to_condition[label]

        conv = conv_templates["mistral_instruct"].copy()
        conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
        conv.append_message(conv.roles[1], answer)
        conversation = conv.get_prompt()

        # Tokenize the conversation
        input_ids = tokenizer_image_token(conversation, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')

        # Create labels - mask the question part, only train on the answer
        labels = input_ids.clone()

        # Find where the assistant's response starts
        assistant_start_token = self.tokenizer.encode(conv.roles[1], add_special_tokens=False)[0]
        try:
            assistant_idx = (input_ids == assistant_start_token).nonzero(as_tuple=True)[0]
            if len(assistant_idx) > 0:
                # Mask everything before the assistant's response
                labels[:assistant_idx[0]] = -100
        except:
            # If we can't find the assistant token, mask first half as fallback
            labels[:len(labels)//2] = -100

        # Load and process image
        image_path = os.path.join(self.image_dir, filename)
        try:
            image = Image.open(image_path).convert("RGB")
            pixel_values = self.image_processor.preprocess(image, return_tensors="pt")["pixel_values"][0]
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
            # Create a blank image tensor
            pixel_values = torch.zeros((3, 224, 224))

        return {
            "input_ids": input_ids.flatten(),
            "labels": labels.flatten(),
            "pixel_values": pixel_values,
        }

In [4]:
train_dataset = MultiClassChestXrayDataset(
    "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/train",
    "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/train.csv",
    "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/pneumothorax_combined_open_classification.csv",
    tokenizer, image_processor
)

val_dataset = MultiClassChestXrayDataset(
    "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/val",
    "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/val.csv",
    "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/pneumothorax_combined_open_classification.csv",
    tokenizer, image_processor
)

def collate_fn(instances):
    input_ids = [instance['input_ids'] for instance in instances]
    labels = [instance['labels'] for instance in instances]
    images = [instance['pixel_values'] for instance in instances]

    # Pad sequences to the same length
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    labels = torch.nn.utils.rnn.pad_sequence(
        labels, batch_first=True, padding_value=-100
    )
    images = torch.stack(images, dim=0)

    return {
        'input_ids': input_ids,
        'labels': labels,
        'images': images,
        'attention_mask': input_ids.ne(tokenizer.pad_token_id),
    }

Dataset initialized with 8497 valid samples
Dataset initialized with 1817 valid samples


In [5]:
from transformers import Trainer

class LlavaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        try:
            # Extract images (not pixel_values for LLaVA-Med)
            images = inputs.pop("images", None)

            # Forward pass with correct parameter name
            outputs = model(
                input_ids=inputs['input_ids'],
                attention_mask=inputs.get('attention_mask', None),
                labels=inputs['labels'],
                images=images
            )

            loss = outputs.loss
            return (loss, outputs) if return_outputs else loss

        except Exception as e:
            print(f"Error in compute_loss: {e}")
            print(f"Input shapes: input_ids={inputs['input_ids'].shape}, labels={inputs['labels'].shape}")
            if 'images' in inputs:
                print(f"images shape: {inputs['images'].shape}")
            raise e


In [6]:
import os
# Set environment variables for better debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

training_args = TrainingArguments(
    output_dir="./lora_output",
    per_device_train_batch_size=2,  # Increased batch size slightly
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,   # Added gradient accumulation
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    logging_strategy="steps",
    logging_steps=10,
    logging_dir="./logs",
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=50,
    fp16=True,
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    group_by_length=False,  # Disable to avoid issues with variable length sequences
)

trainer = LlavaTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn,
)

# Start training
trainer.train()


Step,Training Loss,Validation Loss
50,0.071900,0.073022
100,1.336100,3.637133
150,0.134600,0.077036
200,0.070900,0.068508
250,0.069800,0.070088
300,0.067200,0.067590
350,0.067200,0.066679
400,0.065400,0.066202
450,0.067000,0.066214
500,0.066000,0.065726


Checkpoint destination directory ./lora_output/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Step,Training Loss,Validation Loss
50,0.071900,0.073022
100,1.336100,3.637133
150,0.134600,0.077036
200,0.070900,0.068508
250,0.069800,0.070088
300,0.067200,0.067590
350,0.067200,0.066679
400,0.065400,0.066202
450,0.067000,0.066214
500,0.066000,0.065726


TrainOutput(global_step=1062, training_loss=0.10361526249828985, metrics={'train_runtime': 13967.2493, 'train_samples_per_second': 0.608, 'train_steps_per_second': 0.076, 'total_flos': 3.786764568813158e+16, 'train_loss': 0.10361526249828985, 'epoch': 1.0})

In [7]:
model.save_pretrained("/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)")
tokenizer.save_pretrained("/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)")


('/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)/tokenizer_config.json',
 '/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)/special_tokens_map.json',
 '/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)/tokenizer.model',
 '/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)/added_tokens.json',
 '/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)/tokenizer.json')

# Inference

In [8]:
import os
import random
import pandas as pd
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from llava.mm_utils import tokenizer_image_token
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN

# Label mapping for evaluation
condition_to_label = {
    "no finding": 0,
    "pneumothorax": 1,
    "cardiomegaly": 2,
    "infiltration": 3,
    "effusion": 4,
    "emphysema": 5,
    "atelectasis": 6
}

label_to_condition = {
    0: "No Finding",
    1: "Pneumothorax",
    2: "Cardiomegaly",
    3: "Infiltration",
    4: "Effusion",
    5: "Emphysema",
    6: "Atelectasis"
}

def parse_response_to_label(response):
    """Parse model response to extract predicted label"""
    response_lower = response.lower().strip()

    # Direct mapping first
    if response_lower in condition_to_label:
        return condition_to_label[response_lower]

    # Partial matching
    for condition, label in condition_to_label.items():
        if condition in response_lower:
            return label

    # Default fallback - try to find any condition mentioned
    for condition in condition_to_label.keys():
        if condition.replace(" ", "") in response_lower.replace(" ", ""):
            return condition_to_label[condition]

    # If nothing matches, return -1 to indicate parsing failure
    return -1

def generate_response(image_path, metadata=None):
    try:
        image = Image.open(image_path).convert('RGB')
        image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values']

        # Move to same device as model
        if hasattr(model, 'device'):
            image_tensor = image_tensor.to(model.device, dtype=torch.float32)
        else:
            image_tensor = image_tensor.to('cuda', dtype=torch.float32)

        # Create the same prompt format as training
        age = metadata.get('Patient Age', 'unknown') if metadata else 'unknown'
        sex = metadata.get('Patient Sex', 'unknown') if metadata else 'unknown'

        question = f"You are an expert radiologist. This chest X-ray is from a {age}-year-old {sex} patient. What condition does this chest X-ray show? Choose from: No Finding, Pneumothorax, Cardiomegaly, Infiltration, Effusion, Emphysema, or Atelectasis."

        # Use the exact same conversation format as training
        conv = conv_templates["mistral_instruct"].copy()
        conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
        conv.append_message(conv.roles[1], None)
        prompt_formatted = conv.get_prompt()

        # Tokenize using the same method as training
        input_ids = tokenizer_image_token(prompt_formatted, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')

        # Ensure proper shape
        if input_ids.dim() == 1:
            input_ids = input_ids.unsqueeze(0)

        input_ids = input_ids.to(model.device)

        with torch.inference_mode():
            output_ids = model.base_model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=0.2,
                top_p=0.9,
                max_new_tokens=64,  # Reduced since we expect short responses
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

        # Decode only the new tokens
        input_token_len = input_ids.shape[1]

        # Handle case where output might be shorter than input
        if output_ids.shape[1] > input_token_len:
            outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
        else:
            outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
            # Remove the input part if it's still there
            if question in outputs:
                outputs = outputs.split(question)[-1]

        return outputs.strip()

    except Exception as e:
        print(f"Error in generate_response: {e}")
        return "Error generating response"

In [9]:
from llava.model.builder import load_pretrained_model
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/lora_open_output/model (Open 1 Epoch)")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlavaMistralForCausalLM(
      (model): LlavaMistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=1024, bias=False
   

In [11]:
import torch

test_folder = "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/test"
csv_path = f"/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/test.csv"
metadata_csv_path = "/content/drive/MyDrive/preprocessed_pneumothorax_open_classification/pneumothorax_combined_open_classification.csv"

df = pd.read_csv(csv_path)
filename_to_label = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
metadata_df = pd.read_csv(metadata_csv_path)
metadata_lookup = metadata_df.set_index("Image Index")[["Patient Age", "Patient Sex"]].to_dict(orient="index")

true_labels, predicted_labels = [], []
parse_failures = 0
shuffled_items = list(filename_to_label.items())
random.seed(42)
random.shuffle(shuffled_items)
model = model.float().eval()

for i, (filename, label) in enumerate(shuffled_items):
    image_path = os.path.join(test_folder, filename)
    if not os.path.exists(image_path):
        continue

    metadata = metadata_lookup.get(filename)
    try:
        response = generate_response(image_path, metadata)
        predicted_label = parse_response_to_label(response)

        if predicted_label == -1:
            parse_failures += 1
            print(f"Failed to parse response: '{response}' for {filename}")
            continue

        true_labels.append(int(label))
        predicted_labels.append(predicted_label)

        print(f"Filename       : {filename}")
        print(f"Ground Truth   : {label_to_condition[int(label)]}")
        print(f"Model Response : {response}")
        print(f"Parsed Label   : {label_to_condition[predicted_label]}")
        print("-" * 60)

        if (i + 1) % 20 == 0:
            # Calculate metrics
            total = len(true_labels)
            correct = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p])
            accuracy = correct / total if total > 0 else 0

            # Use macro averaging for multi-class
            precision_macro = precision_score(true_labels, predicted_labels, average='macro', zero_division=0)
            recall_macro = recall_score(true_labels, predicted_labels, average='macro', zero_division=0)
            f1_macro = f1_score(true_labels, predicted_labels, average='macro', zero_division=0)

            print(f"=== INTERMEDIATE RESULTS ({i + 1} samples processed, {total} valid) ===")
            print(f"Parse failures: {parse_failures}")
            print(f"Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")
            print(f"Macro Precision: {precision_macro:.4f}")
            print(f"Macro Recall: {recall_macro:.4f}")
            print(f"Macro F1-Score: {f1_macro:.4f}")
            print("=" * 60)

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        continue

# === FINAL METRICS ===
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score,
    balanced_accuracy_score, matthews_corrcoef, classification_report
)

if true_labels and predicted_labels:
    total = len(true_labels)
    correct = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p])
    accuracy = correct / total

    # Multi-class metrics
    precision_macro = precision_score(true_labels, predicted_labels, average='macro', zero_division=0)
    precision_micro = precision_score(true_labels, predicted_labels, average='micro', zero_division=0)
    precision_weighted = precision_score(true_labels, predicted_labels, average='weighted', zero_division=0)

    recall_macro = recall_score(true_labels, predicted_labels, average='macro', zero_division=0)
    recall_micro = recall_score(true_labels, predicted_labels, average='micro', zero_division=0)
    recall_weighted = recall_score(true_labels, predicted_labels, average='weighted', zero_division=0)

    f1_macro = f1_score(true_labels, predicted_labels, average='macro', zero_division=0)
    f1_micro = f1_score(true_labels, predicted_labels, average='micro', zero_division=0)
    f1_weighted = f1_score(true_labels, predicted_labels, average='weighted', zero_division=0)

    balanced_acc = balanced_accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)

    print("\n=== FINAL ACCURACY RESULTS ===")
    print(f"Total samples processed: {len(shuffled_items)}")
    print(f"Valid predictions: {total}")
    print(f"Parse failures: {parse_failures}")
    print(f"Correct predictions: {correct}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")

    print("\n=== CONFUSION MATRIX ===")
    cm = confusion_matrix(true_labels, predicted_labels, labels=list(range(7)))
    print("Confusion Matrix (rows=true, cols=predicted):")
    print("Classes: 0=No Finding, 1=Pneumothorax, 2=Cardiomegaly, 3=Infiltration, 4=Effusion, 5=Emphysema, 6=Atelectasis")
    print(cm)

    print("\n=== METRICS ===")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Precision (Weighted): {precision_weighted:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"Recall (Weighted): {recall_weighted:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print(f"F1-Score (Weighted): {f1_weighted:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"MCC: {mcc:.4f}")

    print("\n=== CLASS-WISE STATISTICS ===")
    # Count samples per class in true labels
    from collections import Counter
    true_counts = Counter(true_labels)
    pred_counts = Counter(predicted_labels)

    print("True Label Distribution:")
    for label in range(7):
        count = true_counts.get(label, 0)
        percentage = (count / len(true_labels)) * 100 if len(true_labels) > 0 else 0
        print(f"  {label_to_condition[label]}: {count} samples ({percentage:.1f}%)")

    print("\nPredicted Label Distribution:")
    for label in range(7):
        count = pred_counts.get(label, 0)
        percentage = (count / len(predicted_labels)) * 100 if len(predicted_labels) > 0 else 0
        print(f"  {label_to_condition[label]}: {count} samples ({percentage:.1f}%)")

    # Per-class precision, recall, f1
    print("\nPer-Class Performance:")
    precision_per_class = precision_score(true_labels, predicted_labels, average=None, zero_division=0, labels=range(7))
    recall_per_class = recall_score(true_labels, predicted_labels, average=None, zero_division=0, labels=range(7))
    f1_per_class = f1_score(true_labels, predicted_labels, average=None, zero_division=0, labels=range(7))

    for label in range(7):
        tp = sum([1 for t, p in zip(true_labels, predicted_labels) if t == label and p == label])
        fn = sum([1 for t, p in zip(true_labels, predicted_labels) if t == label and p != label])
        fp = sum([1 for t, p in zip(true_labels, predicted_labels) if t != label and p == label])
        tn = sum([1 for t, p in zip(true_labels, predicted_labels) if t != label and p != label])

        accuracy_class = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

        print(f"\n{label_to_condition[label]} (Label {label}):")
        print(f"  True Positives: {tp}")
        print(f"  False Positives: {fp}")
        print(f"  False Negatives: {fn}")
        print(f"  True Negatives: {tn}")
        print(f"  Class Accuracy: {accuracy_class:.4f} ({accuracy_class * 100:.2f}%)")
        print(f"  Precision: {precision_per_class[label]:.4f} ({precision_per_class[label] * 100:.2f}%)")
        print(f"  Recall: {recall_per_class[label]:.4f} ({recall_per_class[label] * 100:.2f}%)")
        print(f"  F1-Score: {f1_per_class[label]:.4f} ({f1_per_class[label] * 100:.2f}%)")

    print("\n=== DETAILED CLASSIFICATION REPORT ===")
    target_names = [label_to_condition[i] for i in range(7)]
    print(classification_report(true_labels, predicted_labels, target_names=target_names, zero_division=0))

Streaming output truncated to the last 5000 lines.
=== INTERMEDIATE RESULTS (920 samples processed, 920 valid) ===
Parse failures: 0
Accuracy: 0.3848 (38.48%)
Macro Precision: 0.3999
Macro Recall: 0.3514
Macro F1-Score: 0.3493
Filename       : 00028186_002.png
Ground Truth   : Infiltration
Model Response : Atelectasis
Parsed Label   : Atelectasis
------------------------------------------------------------
Filename       : 00015170_006.png
Ground Truth   : Atelectasis
Model Response : Pneumothorax
Parsed Label   : Pneumothorax
------------------------------------------------------------
Filename       : 00017138_047.png
Ground Truth   : Effusion
Model Response : Atelectasis
Parsed Label   : Atelectasis
------------------------------------------------------------
Filename       : 00016964_006.png
Ground Truth   : Atelectasis
Model Response : Effusion
Parsed Label   : Effusion
------------------------------------------------------------
Filename       : 00017972_021.png
Ground Truth   : 